In [3]:
# grid search ets models
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from pandas import read_csv
from numpy import array
import pandas as pd
import numpy as np
import pandas.util.testing as tm

In [4]:
# one-step Holt Winter’s Exponential Smoothing forecast
def exp_smoothing_forecast(history, config):
	t,d,s,p,b,r = config
	# define model
	history = array(history)
	model = ExponentialSmoothing(history, trend=t, damped=d, seasonal=s, seasonal_periods=p)
	# fit model
	model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]
 
# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))
 
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]
 
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = exp_smoothing_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error
 
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)
 
# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores
 
# create a set of exponential smoothing configs to try
def exp_smoothing_configs(seasonal=[None]):
	models = list()
	# define config lists
	t_params = ['add', 'mul', None]
	d_params = [True, False]
	s_params = ['add', 'mul', None]
	p_params = seasonal
	b_params = [True, False]
	r_params = [True, False]
	# create config instances
	for t in t_params:
		for d in d_params:
			for s in s_params:
				for p in p_params:
					for b in b_params:
						for r in r_params:
							cfg = [t,d,s,p,b,r]
							models.append(cfg)
	return models
 


In [5]:
path = '/content/drive/MyDrive/Allemand/county_daily.csv'
data = pd.read_csv(path, parse_dates=True)
data.head()  
df3 = data.groupby("date").agg(list) # dataframe with date as index and each columns contain list

# Number of cases

In [ ]:
df3['cases'][0]
total_cases = []
for i in range(852) :
    total_cases.append(sum(df3['cases'][i]))

df3['total_cases'] = total_cases
for i in range(852) :
    df3.loc[df3['total_cases'] == 0, 'B'] = 0.00001
    df3.loc[df3['total_cases'] < 0, 'B'] = 0.00001
    df3.loc[df3['total_cases'] > 0, 'B'] = df3['total_cases']

df3.index.freq='D'

data2 = []
for i in range(len(df3['B'])) : 
  data2.append(np.array(df3['B'][i]))

In [25]:
if __name__ == '__main__':
	# load dataset
	data = np.array(data2)
	# data split
	n_test = 170
	# model configs
	cfg_list = exp_smoothing_configs()
	# grid search
	scores = grid_search(data, cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

 > Model[['add', True, None, None, False, True]] 174437.200
 > Model[['add', True, None, None, True, True]] 170165.866
 > Model[['add', True, None, None, False, False]] 174274.146
 > Model[['add', True, None, None, True, False]] 170373.397
 > Model[['add', False, None, None, True, True]] 173611.810
 > Model[['add', False, None, None, True, False]] 173545.444
 > Model[['add', False, None, None, False, True]] 178843.248
 > Model[['add', False, None, None, False, False]] 178631.762
 > Model[[None, False, None, None, True, True]] 177528.426
 > Model[[None, False, None, None, False, True]] 177658.725
 > Model[[None, False, None, None, True, False]] 178179.608
 > Model[[None, False, None, None, False, False]] 177498.829
done
['add', True, None, None, True, True] 170165.8657395654
['add', True, None, None, True, False] 170373.39672161834
['add', False, None, None, True, False] 173545.44398144458


# Number of deaths

In [7]:
df3['deaths'][0]
total_deaths = []
for i in range(852) :
    total_deaths.append(sum(df3['deaths'][i]))

df3['total_deaths'] = total_deaths

df3.loc[df3['total_deaths'] == 0, 'total_deaths'] = 0.00001
df3.loc[df3['total_deaths'] < 0, 'total_deaths'] = 0.00001


df3.index.freq='D'

data3 = []
for i in range(len(df3['total_deaths'])) : 
  data3.append(np.array(df3['total_deaths'][i]))

In [26]:
if __name__ == '__main__':
	# load dataset
	data3 = np.array(data3)
	# data split
	n_test = 170
	# model configs
	cfg_list = exp_smoothing_configs()
	# grid search
	scores = grid_search(data3, cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

 > Model[['add', True, None, None, False, True]] 1057.055
 > Model[['add', True, None, None, False, False]] 1056.513
 > Model[['add', True, None, None, True, False]] 1074.802
 > Model[['add', False, None, None, True, False]] 1347.648
 > Model[['add', False, None, None, False, True]] 1059.704
 > Model[['add', False, None, None, False, False]] 1059.709
 > Model[[None, False, None, None, True, True]] 1081.052
 > Model[[None, False, None, None, True, False]] 1097.050
 > Model[[None, False, None, None, False, True]] 1059.847
 > Model[[None, False, None, None, False, False]] 1058.830
done
['add', True, None, None, False, False] 1056.5129628579614
['add', True, None, None, False, True] 1057.0551615192633
[None, False, None, None, False, False] 1058.8302764543414
